In [1]:
import sys

import pandas as pd

sys.path.append('../../src/construct/')
import util

idx = pd.IndexSlice

In [2]:
# HRE data
HRE_dfs = []
for k, v in {
    'household': dict(sheet_name='Residential data', skiprows=6, index_col=[0, 1], usecols="F,K,V:X"), 
    'commercial': dict(sheet_name='Tertiary data', skiprows=4, index_col=[0, 1], usecols="F,L,O:Q"),
    'industry': dict(sheet_name='Industry data', skiprows=7, index_col=[0, 1], usecols="N,S,X:AB")
}.items():
    _df = pd.read_excel('https://heatroadmap.eu/wp-content/uploads/2018/09/HRE4-Exchange-Template-WP3_v22b_website.xlsx', **v)
    if k == 'household':
        _df.columns = pd.Index(['water_heating', 'space_heating', 'cooking'], name='end_use')  # Assume HRE 'other heating' is just cooking
    elif k == 'commercial':
        _df.columns = pd.Index(['water_heating', 'space_heating', 'process_heating'], name='end_use')
    elif k == 'industry':
        _df = _df.groupby({
            'PH <100 (FED)': 'process_heating',
            'PH 100-200 (FED)': 'process_heating',
            'PH 200-500 (FED)': 'process_heating',
            'PH >500 (FED)': 'process_heating',
            'SH (FED)': 'space_heating'
        }, axis=1).sum().rename_axis(columns='end_use')

    _df.index = (
        _df.index.set_names(['country_code', 'carrier_name'])
    )

    _df = (
        _df
        .apply(util.to_numeric)
        .dropna(how='all')
        .groupby([
            {i: util.get_alpha3(i) for i in _df.index.levels[0]},
            {'Biomass': 'biomass',
            'Coal': 'solid_fossil',
            'Compression cooling (electricity)': 'electricity',
            'District heating': 'heat',
            'Electric Heating': 'electricity',
            'Gas': 'natural_gas',
            'Heat pumps total (electric)': 'electricity',
            'Micro CHP (biomass)': 'biomass',
            'Micro CHP (natural gas)': 'natural_gas',
            'Oil': 'oil',
            'Others (RES)': 'other_renewable',
            'Others (fossil)': 'solid_fossil',  # This is where things like 'peat' and shale sand oil has gone from Eurostat...
            'Solar thermal': 'solar_thermal'},
        ], level=[0, 1]).sum()
        .assign(cat_name=k)
        .set_index('cat_name', append=True)
        .stack()
        .reorder_levels(['country_code', 'cat_name', 'end_use', 'carrier_name'])
    )
    HRE_dfs.append(_df)
HRE_df = pd.concat(HRE_dfs)

In [3]:
HRE_df.to_csv('../data/industry/FORECAST-industry_2015_end_use_consumption.csv')